In [ ]:
# Evaluate PanopticFPN on COCO panoptic val2017 (MMDetection 3.x)
import os
from mmengine.config import Config
from mmengine.runner import Runner

# ---- 1) Paths ----
CONFIG = r'C:/Users/heheh/mmdetection/configs/panoptic_fpn/panoptic-fpn_r50_fpn_1x_coco.py'
CHECKPOINT = r'C:/Users/heheh/mmdetection/checkpoints/panoptic_fpn_r50_fpn_1x_coco_20210821_101153-9668fd13.pth'
DATA_ROOT = r'C:/Users/heheh/mmdetection/data/coco'
PAN_JSON  = os.path.join(DATA_ROOT, 'annotations', 'panoptic_val2017.json')
IMG_DIR   = os.path.join(DATA_ROOT, 'val2017')                   # same images as instance seg
PAN_SEG   = os.path.join(DATA_ROOT, 'annotations', 'panoptic_val2017')  # PNG maps
WORK_DIR  = './work_dirs/eval_panoptic_fpn'

# ---- 2) Load cfg and override dataset/evaluator ----
cfg = Config.fromfile(CONFIG)
cfg.default_scope = 'mmdet'
cfg.load_from = CHECKPOINT
cfg.work_dir = WORK_DIR

# dataloader settings (adjust if you like)
cfg.test_dataloader.batch_size = 2
cfg.test_dataloader.num_workers = 2

# Point test dataset to COCO Panoptic format
ds = cfg.test_dataloader.dataset
ds.type = 'CocoPanopticDataset'              # <-- panoptic dataset type
ds.data_root = DATA_ROOT
ds.ann_file = PAN_JSON                       # <-- panoptic json
ds.data_prefix = dict(img=IMG_DIR, seg=PAN_SEG)  # <-- img dir + seg png dir
ds.test_mode = True

# Evaluator: COCO panoptic metric (PQ/SQ/RQ)
cfg.test_evaluator = dict(
    type='CocoPanopticMetric',
    ann_file=PAN_JSON, 
    seg_prefix=PAN_SEG,# GT panoptic json
    format_only=False          # compute metrics instead of just dumping results
)

# If your config defines a visualizer, disable disk writing during test
if hasattr(cfg, 'visualizer'):
    cfg.visualizer.vis_backends = None

# ---- 3) Run evaluation ----
os.makedirs(cfg.work_dir, exist_ok=True)
runner = Runner.from_cfg(cfg)
metrics = runner.test()
print(metrics)
